# Scraping Rotten Tomatoes 140 ESSENTIAL ACTION MOVIES TO WATCH NOW

In [14]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

In [2]:
base_url = 'https://editorial.rottentomatoes.com/guide/140-essential-action-movies-to-watch-now/'

response = requests.get(base_url)

if response.ok:
    print('Request OK')
else:
    print('Request failed')

# Obtain the html binary from the reponse
html = response.content

# Makse the soup
soup = BeautifulSoup(html, 'lxml')

# Save the html
with open('top_140_action_movie_rotten_tomatoes.html', 'wb') as file:
    file.write(soup.prettify('utf-8'))

Request OK


Inspecting the website is easily noticed that the name, year, cricitcs consensus, 
synops, cast and director ate in the div element with the class = "col-sm-18 col-full-xs countdown-item-content"

In [3]:
data_elements = soup.find_all('div', class_ = "col-sm-18 col-full-xs countdown-item-content" )
print(len(data_elements))

140


As we can se the find_all return have 140 elements, the same of the number of the movies in the list.

In [4]:
data_elements[0]

<div class="col-sm-18 col-full-xs countdown-item-content">
<div class="row countdown-item-title-bar">
<div class="col-sm-20 col-full-xs" style="height: 100%;">
<div class="article_movie_title" style="float: left;">
<div><h2><a href="https://www.rottentomatoes.com/m/1018009-running_scared">Running Scared</a> <span class="subtle start-year">(1986)</span> <span class="icon tiny rotten" title="Rotten"></span> <span class="tMeterScore">59%</span></h2></div>
</div>
</div>
<div class="col-sm-4 col-full-xs" style="height: 100%;">
<div class="countdown-index">#140</div>
</div>
</div>
<div class="row countdown-item-details">
<div class="col-sm-24">
<div class="info countdown-adjusted-score"><span class="descriptor">Adjusted Score: </span>55415% <span class="glyphicon glyphicon-question-sign" data-html="true" data-original-title="The Adjusted Score comes from a weighted formula (Bayesian) that we use that accounts for variation in the number of reviews per movie." data-placement="top" data-toggle

#### Extracting the titles 
Printing one of the data_elements and specting the website we can see that the titles are stored in the anchor inside h2

In [5]:
headers2 = [movie_data.find('h2') for movie_data in data_elements]
titles = [title.find('a').text for title in headers2 if title.find('a').text != None]
print(titles)

['Running Scared', 'Equilibrium', 'Hero', 'Road House', 'Unstoppable', 'Shaft', 'The Villainess', 'Highlander', 'Die Hard 2', 'National Treasure', 'The Protector', 'Revenge', 'El Mariachi', 'A Touch of Zen', 'Top Gun', 'Con Air', 'The Expendables 2', 'The Mummy', 'Mr. & Mrs. Smith', 'Rush Hour', 'The Equalizer', 'Captain America: Civil War', 'Air Force One', 'Bloodsport', 'Blade', 'Bad Boys', 'Die Hard With a Vengeance', 'The Running Man', 'Code of Silence', "Shoot 'Em Up", 'Crank', 'Machete', 'Drive', 'Batman', 'Under Siege', 'Independence Day', 'Bullitt', 'Wanted', 'Superman: The Movie', 'Ronin', 'They Live', 'Cliffhanger', "Marvel's the Avengers", 'Hot Fuzz', 'The Warriors', 'Starship Troopers', 'Elite Squad 2', 'Point Break', 'The Long Kiss Goodnight', 'The Guest', 'Taken', '300', 'True Lies', 'Demolition Man', 'Hardcore Henry', 'Police Story', 'Brotherhood of the Wolf', 'Kingsman: The Secret Service', 'The Fifth Element', 'Big Trouble in Little China', 'Atomic Blonde', 'Rumble in 

#### Extracting released years and scores

In [6]:
released_years = [release_year.find('span', class_ = 'subtle start-year').text.strip('()') for release_year in headers2 ]
print(released_years)
scores = [score.find('span', class_='tMeterScore').text for score in headers2]
print(scores)

['1986', '2002', '2002', '1989', '2010', '1971', '2017', '1986', '1990', '2004', '2005', '2017', '1992', '1971', '1986', '1997', '2012', '1999', '2005', '1998', '2014', '2016', '1997', '1988', '1998', '1995', '1995', '1987', '1985', '2007', '2006', '2010', '2011', '1989', '1992', '1996', '1968', '2008', '1978', '1998', '1988', '1993', '2012', '2007', '1979', '1997', '2010', '1991', '1996', '2014', '2008', '2006', '1994', '1993', '2015', '1985', '2001', '2014', '1997', '1986', '2017', '1995', '2004', '1984', '2003', '2004', '1993', '1981', '2000', '2004', '2010', '1992', '1989', '2004', '1986', '2008', '2018', '2017', '1964', '1976', '2017', '1972', '2014', '2003', '1971', '2015', '1990', '1992', '1971', '2014', '2003', '1993', '2018', '2010', '1995', '2002', '2019', '2012', '2002', '2008', '1997', '1985', '2008', '2011', '2011', '1987', '1996', '1987', '2017', '2006', '2017', '1994', '1989', '2014', '1973', '1985', '1982', '2015', '1984', '2000', '2003', '1994', '1994', '1994', '2014',

#### Extracting Critics Consensus, synopsis, cast and director

In [41]:
# Extra info is the element that contain the critics consensus, synopsis, cast members and director
extra_info = soup.find_all('div', class_='col-sm-24')
critics_consensus = [None if critic_consensu.find('div', class_='info critics-consensus') is None else critic_consensu.find('div', class_='info critics-consensus').text for critic_consensu in extra_info]
critics_consensus= [critic_consensu.replace('Critics Consensus: ', '') for critic_consensu in critics_consensus]

In [46]:
# Synopsis
synopsis = [None if synopsi.find('div', class_='info synopsis') is None else synopsi.find('div', class_='info synopsis').text for synopsi in extra_info]
synopsis= [synopsi.replace('Synopsis: ', '') for synopsi in synopsis]
synopsis= [synopsi.replace('[More]', '') for synopsi in synopsis]

# Main Cast members
casts = [None if cast.find('div', class_='info cast') is None else cast.find('div', class_='info cast').text for cast in extra_info]
casts = [cast.replace('\nStarring: ', '') for cast in casts]

# Directors
directors = [None if director.find('div', class_='info director') is None else director.find('div', class_='info director') for director in extra_info]
directors = [None if director.find('a') is None else director.find('a').text for director in directors]

In [50]:
movies_data = {
    'Title': titles,
    'Released Year': released_years,
    'Rotten Tomatoes Score': scores,
    'Cast': casts,
    'Director': directors,
    'Synopsis': synopsis,
    'Critics consensus': critics_consensus
}
movies_df = pd.DataFrame(movies_data)
print(movies_df)
movies_df.to_csv('data_140_action_movies.csv')



                          Title Released Year Rotten Tomatoes Score  \
0                Running Scared          1986                   59%   
1                   Equilibrium          2002                   40%   
2                          Hero          2002                   94%   
3                    Road House          1989                   41%   
4                   Unstoppable          2010                   87%   
..                          ...           ...                   ...   
135                 Hard-Boiled          1992                   95%   
136                  The Matrix          1999                   88%   
137  Terminator 2: Judgment Day          1991                   91%   
138                    Die Hard          1988                   94%   
139          Mad Max: Fury Road          2015                   97%   

                                                  Cast          Director  \
0    Gregory Hines, Billy Crystal, Jimmy Smits, Ste...       Peter Hyam